## Workshop Week 7 (Optional)

### Note
As said in iLearn, the main purpose of this week's workshop is ensure portfolio 2 on the right track. If you are interested, you may complete the following tasks. 

## Logistic Regression
Breast Cancer data from [the UCI repository](http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29) contains records corresponding to 
cases of observed tumors.   There are a number of observations for each and a categorisation in the `class` column: 2 for benign (good), 4 for malignant (bad).  Your task is to build a logistic regression model to classify these cases. 

The data is provided as a CSV file.  There are a small number of cases where no value is available, these are indicated in the data with `?`. I have used the `na_values` keyword for `read_csv` to have these interpreted as `NaN` (Not a Number).  Your first task is to decide what to do with these rows. You could just drop these rows or you could [impute them from the other data](http://scikit-learn.org/stable/modules/preprocessing.html#imputation-of-missing-values).

You then need to follow the procedure outlined in the lecture for generating a train/test set, building and evaluating a model. Your goal is to build the best model possible over this data.   Your first step should be to build a logistic regression model using all of the features that are available.
  

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_selection import RFE

In [3]:
bcancer = pd.read_csv("breast-cancer-wisconsin.csv", na_values="?")
bcancer.head()

FileNotFoundError: [Errno 2] No such file or directory: 'breast-cancer-wisconsin.csv'

In [ ]:
# Examine the data: check number of rows and number of columns
print(f"Number of rows and columns: {bcancer.shape}")

In [ ]:
# Look at the statistical summary of the dataframe
print(bcancer.describe())

In [ ]:
# Check how many classes we do have from the "class" column
print(bcancer['class'].value_counts())

In [ ]:
# Check number of samples for each class and comment whether dataset is balanced?
class_counts = bcancer['class'].value_counts()

print(f"Number of samples for each class:\n{class_counts}")

# Calculate the percentage of each class
class_percentages = class_counts / len(bcancer) * 100
print(f"Class percentages:\n{class_percentages}")

# Comment: If one class is significantly larger than the other, the dataset is imbalanced.
if abs(class_percentages[0] - class_percentages[1]) > 10:
    print("The dataset is imbalanced.")
else:
    print("The dataset is relatively balanced.")

In [ ]:
# Deal with the NaN values in the data
imputer = SimpleImputer(strategy='mean')
bcancer.iloc[:, :] = imputer.fit_transform(bcancer)

In [ ]:
# Split your data into training(80%) and testing data (20%) and use random_state=142
from sklearn.model_selection import train_test_split

# Separate features and target variable
X = bcancer.drop(columns='class')
y = bcancer['class']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=142)


In [ ]:
# Build your Logistic Regression model
from sklearn.linear_model import LogisticRegression

# Initialize the logistic regression model
model = LogisticRegression(max_iter=10000)

# Train the model
model.fit(X_train, y_train)


In [ ]:
# Do predictions on test set
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
from sklearn.metrics import accuracy_score, confusion_matrix

# Accuracy
accuracy = accuracy_score(y_test, y_pred)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)


### Evaluation

To evaluate a classification model we want to look at how many cases were correctly classified and how many
were in error.  In this case we have two outcomes - benign and malignant.   SKlearn has some useful tools, the 
[accuracy_score]() function gives a score from 0-1 for the proportion correct.  The 
[confusion_matrix](http://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix) function 
shows how many were classified correctly and what errors were made.  Use these to summarise the performance of 
your model (these functions have already been imported above).

In [13]:
# Evaluate the performance of your trained model
from sklearn.metrics import accuracy_score, confusion_matrix

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Breakdown of the confusion matrix
TP = conf_matrix[1, 1]  # True Positives
TN = conf_matrix[0, 0]  # True Negatives
FP = conf_matrix[0, 1]  # False Positives
FN = conf_matrix[1, 0]  # False Negatives

print(f"True Positives: {TP}")
print(f"True Negatives: {TN}")
print(f"False Positives: {FP}")
print(f"False Negatives: {FN}")

# Additional comment:
print("\nIn summary:")
print(f" - {TP} malignant cases correctly classified as malignant.")
print(f" - {TN} benign cases correctly classified as benign.")
print(f" - {FP} benign cases incorrectly classified as malignant.")
print(f" - {FN} malignant cases incorrectly classified as benign.")


NameError: name 'y_test' is not defined

**This is the checkpoint mark for this week's workshop. You need to report `Accuracy Score` on test set and also show `confusion matrix`. You also need to provide analysis based on the results you got.**

### Feature Selection

Since you have many features available, one part of building the best model will be to select which features to use as input to the classifier. Your initial model used all of the features but it is possible that a better model can 
be built by leaving some of them out.   Test this by building a few models with subsets of the features - how do your models perform? 

This process can be automated.  The [sklearn RFE function](http://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination) implements __Recursive Feature Estimation__ which removes 
features one by one, evaluating the model each time and selecting the best model for a target number of features.  Use RFE to select features for a model with 3, 4 and 5 features - can you build a model that is as good or better than your initial model?

In [15]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Initialize the Logistic Regression model
model = LogisticRegression(max_iter=10000)

# Create RFE object to select 3, 4, and 5 features
for n_features in [3, 4, 5]:
    rfe = RFE(estimator=model, n_features_to_select=n_features)
    
    # Fit RFE on the training data
    rfe.fit(X_train, y_train)
    
    # Select the features
    selected_features = X.columns[rfe.support_]
    print(f"Selected {n_features} features: {selected_features.tolist()}")
    
    # Build a logistic regression model using only the selected features
    X_train_rfe = rfe.transform(X_train)
    X_test_rfe = rfe.transform(X_test)
    
    # Train the model with the selected features
    model.fit(X_train_rfe, y_train)
    
    # Make predictions on the test set
    y_pred_rfe = model.predict(X_test_rfe)
    
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred_rfe)
    conf_matrix = confusion_matrix(y_test, y_pred_rfe)
    
    print(f"Accuracy with {n_features} features: {accuracy:.4f}")
    print(f"Confusion Matrix with {n_features} features:\n", conf_matrix)


NameError: name 'X_train' is not defined

## Conclusion 

Write a brief conclusion to your experiment. You might comment on the proportion of __false positive__ and __false negative__ classifications your model makes.  How useful would this model be in a clinical diagnostic setting? 